In [7]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [2]:
news_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
import pandas as pd

news_data.data[0]

'From: egreen@east.sun.com (Ed Green - Pixel Cruncher)\nSubject: Re: Observation re: helmets\nOrganization: Sun Microsystems, RTP, NC\nLines: 21\nDistribution: world\nReply-To: egreen@east.sun.com\nNNTP-Posting-Host: laser.east.sun.com\n\nIn article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:\n> \n> The question for the day is re: passenger helmets, if you don\'t know for \n>certain who\'s gonna ride with you (like say you meet them at a .... church \n>meeting, yeah, that\'s the ticket)... What are some guidelines? Should I just \n>pick up another shoei in my size to have a backup helmet (XL), or should I \n>maybe get an inexpensive one of a smaller size to accomodate my likely \n>passenger? \n\nIf your primary concern is protecting the passenger in the event of a\ncrash, have him or her fitted for a helmet that is their size.  If your\nprimary concern is complying with stupid helmet laws, carry a real big\nspare (you can put a big or small 

In [4]:
len(news_data.data)

18846

[과제] 다음을 수행하세요 
- BOW 피처벡터화 (카운트,tfidf)
- 모델학습/예측/평가(logistic Regression)
- 모델 성능 개선을 위한 튜닝 (GridSearchCV)

In [8]:
train_news =fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=156)
X_train = train_news.data
y_train = train_news.target

test_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
len(train_news.data),len(test_news.data)

(11314, 7532)

In [6]:
# 피처벡터화 수행(카운트 기반의 벡터화)
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


In [7]:
#분류 예측
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr = LogisticRegression()
lr.fit(X_train_cnt_vect , y_train)
pred = lr.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.605


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
#TD-IDF 벡터화 수행
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
x_test_tfidf = tfidf_vect.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_tfidf,y_train)
pred = lr.predict(x_test_tfidf)
print('TF-IDF의 LR 예측 정확도는:',accuracy_score(y_test,pred))

TF-IDF의 LR 예측 정확도는: 0.6736590546999469


In [9]:
#머신러닝 모델 향상 시키는 중요한 2가지 법칙은 최적의 ML알고리즘 선택과 최상의 피처 전처리 수행하는 것
# 앞에서는 TF-IDF 벡터화는 기본 파라미터만 적용했지만,좀 더 다양한 파라미터를 적용해 보겠다
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words ='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_train)

lr = LogisticRegression()
lr.fit(X_train_tfidf,y_train)
pred = lr.predict(X_test_tfidf)
print('TF-IDF의 정확도:',accuracy_score(y_test,pred))

MemoryError: Unable to allocate 3.51 GiB for an array with shape (471728180,) and data type float64

In [ ]:
#최적의 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV
params={'C':[0.01,0.1,1,5,10]}
grid = GridSearchCV(lr,param_grid= params,cv=3,scoring='accuracy',verbose=1)
grid.fit(X_train_tfidf,y_train)
print('lg best C params:',gird.best_params_)
pred = grid.predict(x_test_tfidf)
print('TF-IDF의 LR 정확도:',accuracy_score(y_test,pred))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline= Pipeline([('tfidf_vect',TfidfVectorizer(stop_words ='english',ngram_range=(1,2),max_df=300)),
                   ('lr_clf',LogisticRegression(C=10))])
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print('pipeline을 통한 lr예측 정확도는:',accuracy_score(y_test,pred))

MemoryError: Unable to allocate 3.51 GiB for an array with shape (471728180,) and data type float64

In [13]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all',random_state=0)
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [15]:
import pandas as pd
print('target 클래스의 값과 분포', pd.Series(news.target).value_counts().sort_index())
print(news.target_names)

target 클래스의 값과 분포 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
from sklearn.datasets import fetch20_newsgroups
train_news = fetch_20newsgroups(subset='train',remove=('header','footer','quotes'),random_state=100)
X_train = train_news.data
y_target = train_news.target
test_news = fetch_20newsgroupw(subset='test',remove=('header','footer','quotes'),random_state=100)
X_test = test_news.data
y_test =test_news.target

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transfrom(X_train)
X_test_cnt_vect = cnt_vect.transfrom(X_test)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
lr.fit(X_train_cnt_vect,y_train)
pred = lr.predict(X_test_cnt_vect)
print(accuracy_score(y_test,pred))

tfidf_vect =TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transfrom(X_train)
X_test_tfidf_vect = tfidf_vect.transfrom(X_test)

lr = LogisticRegression()
lr.fit(X_train_tfidf_vect,y_train)
pred = lr.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,pred))

#stop words 불용어 제거 ngram_range로 단어 개수 불러오는 조건추가
tfidf_vect = TfidVectorizer(stop_words ='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_tfidf_vect,y_train)
pred = lr.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,pred))

from sklearn.model_selection import GridSearchCV

params ={'C':[0.01,0.1,1,5,10]}

grid_lr = GridSearchCV(lr,param_grid=params,cv=3,)

In [ ]:
F